# Optimal Power Flow in Julia




In [ ]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.7.1" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia PowerModels Ipopt"
JULIA_PACKAGES_IF_GPU="CUDA" # or CuArrays for older Julia versions
JULIA_NUM_THREADS=2
#---------------------------------------------------#

if [ -n "$COLAB_GPU" ] && [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  if [ "$COLAB_GPU" = "1" ]; then
      JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia  

  echo ''
  echo "Successfully installed `julia -v`!"
  echo "Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then"
  echo "jump to the 'Checking the Installation' section."
fi

Installing Julia 1.7.1 on the current Colab Runtime...
2022-06-04 15:30:12 URL:https://storage.googleapis.com/julialang2/bin/linux/x64/1.7/julia-1.7.1-linux-x86_64.tar.gz [123374573/123374573] -> "/tmp/julia.tar.gz" [1]
Installing Julia package IJulia...
Installing Julia package PowerModels...
Installing Julia package Ipopt...


# Checking the Installation
The `versioninfo()` function should print your Julia version and some other info about the system:

In [1]:
versioninfo()

Julia Version 1.7.1
Commit ac5cc99908 (2021-12-22 19:35 UTC)
Platform Info:
  OS: Linux (x86_64-pc-linux-gnu)
  CPU: Intel(R) Xeon(R) CPU @ 2.20GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-12.0.1 (ORCJIT, broadwell)
Environment:
  JULIA_NUM_THREADS = 2


In [ ]:
#using Pkg
Pkg.add("PowerModels")

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.7/Project.toml`
  No Changes to `~/.julia/environments/v1.7/Manifest.toml`


In [ ]:
using PowerModels
using Ipopt

In [ ]:
# load the network data
network_data = PowerModels.parse_file("case9.m")

[warn | PowerModels]: this code only supports angmin values in -90 deg. to 90 deg., tightening the value on branch 8 from -360.0 to -60.0 deg.
[warn | PowerModels]: this code only supports angmax values in -90 deg. to 90 deg., tightening the value on branch 8 from 360.0 to 60.0 deg.
[warn | PowerModels]: this code only supports angmin values in -90 deg. to 90 deg., tightening the value on branch 4 from -360.0 to -60.0 deg.
[warn | PowerModels]: this code only supports angmax values in -90 deg. to 90 deg., tightening the value on branch 4 from 360.0 to 60.0 deg.
[warn | PowerModels]: this code only supports angmin values in -90 deg. to 90 deg., tightening the value on branch 1 from -360.0 to -60.0 deg.
[warn | PowerModels]: this code only supports angmax values in -90 deg. to 90 deg., tightening the value on branch 1 from 360.0 to 60.0 deg.
[warn | PowerModels]: this code only supports angmin values in -90 deg. to 90 deg., tightening the value on branch 5 from -360.0 to -60.0 deg.
[warn

Dict{String, Any} with 13 entries:
  "bus"            => Dict{String, Any}("8"=>Dict{String, Any}("zone"=>1, "bus_…
  "source_type"    => "matpower"
  "name"           => "case9"
  "dcline"         => Dict{String, Any}()
  "source_version" => "2"
  "gen"            => Dict{String, Any}("1"=>Dict{String, Any}("ncost"=>3, "qc1…
  "branch"         => Dict{String, Any}("8"=>Dict{String, Any}("br_r"=>0.032, "…
  "storage"        => Dict{String, Any}()
  "switch"         => Dict{String, Any}()
  "baseMVA"        => 100
  "per_unit"       => true
  "shunt"          => Dict{String, Any}()
  "load"           => Dict{String, Any}("1"=>Dict{String, Any}("source_id"=>Any…

In [ ]:
# explore the network data a bit
network_data["branch"]["1"]

Dict{String, Any} with 19 entries:
  "br_r"        => 0.0
  "rate_a"      => 2.5
  "shift"       => 0.0
  "rate_b"      => 2.5
  "br_x"        => 0.0576
  "rate_c"      => 2.5
  "g_to"        => 0.0
  "g_fr"        => 0.0
  "source_id"   => Any["branch", 1]
  "b_fr"        => 0.0
  "f_bus"       => 1
  "br_status"   => 1
  "t_bus"       => 4
  "b_to"        => 0.0
  "index"       => 1
  "angmin"      => -1.0472
  "angmax"      => 1.0472
  "transformer" => false
  "tap"         => 1.0

In [ ]:
# calculate the admittance matrix
Y = PowerModels.calc_admittance_matrix(network_data)

# show the admittance matrix (additonal code to show matrix in full)
show(IOContext(stdout, :limit => true, :displaysize => (500, 500)), "text/plain", Y.matrix)

9×9 SparseArrays.SparseMatrixCSC{ComplexF64, Int64} with 27 stored entries:
  0.0-17.3611im       ⋅            ⋅              -0.0+17.3611im           ⋅                   ⋅                   ⋅                   ⋅                   ⋅    
      ⋅           0.0-16.0im       ⋅                   ⋅                   ⋅                   ⋅                   ⋅              -0.0+16.0im              ⋅    
      ⋅               ⋅        0.0-17.0648im           ⋅                   ⋅              -0.0+17.0648im           ⋅                   ⋅                   ⋅    
 -0.0+17.3611im       ⋅            ⋅           3.30738-39.3089im  -1.94219+10.5107im           ⋅                   ⋅                   ⋅          -1.36519+11.6041im
      ⋅               ⋅            ⋅          -1.94219+10.5107im    3.2242-15.8409im  -1.28201+5.58824im           ⋅                   ⋅                   ⋅    
      ⋅               ⋅       -0.0+17.0648im           ⋅          -1.28201+5.58824im    2.4371-32.1539im  -1.15509+

In [ ]:
# Run an AC-OPF
ac_results = solve_ac_opf(network_data, Ipopt.Optimizer)

This is Ipopt version 3.14.4, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:      223
Number of nonzeros in inequality constraint Jacobian.:       72
Number of nonzeros in Lagrangian Hessian.............:      399

Total number of variables............................:       60
                     variables with only lower bounds:        0
                variables with lower and upper bounds:       51
                     variables with only upper bounds:        0
Total number of equality constraints.................:       55
Total number of inequality constraints...............:       36
        inequality constraints with only lower bounds:        9
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:       27

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  1.2026175e+03 1.25e+00 1.19e+01  -1.0 0.00e+00    -  0.00e+00 0.00e+00  

Dict{String, Any} with 8 entries:
  "solve_time"         => 0.032784
  "optimizer"          => "Ipopt"
  "termination_status" => LOCALLY_SOLVED
  "dual_status"        => FEASIBLE_POINT
  "primal_status"      => FEASIBLE_POINT
  "objective"          => 5296.69
  "solution"           => Dict{String, Any}("baseMVA"=>100, "branch"=>Dict{Stri…
  "objective_lb"       => -Inf

In [ ]:
# explore some results
ac_results["solution"]["branch"]["1"]

Dict{String, Any} with 4 entries:
  "qf" => 0.129656
  "qt" => -0.0904698
  "pt" => -0.897987
  "pf" => 0.897987

In [ ]:
# run a dc opf
dc_results = solve_dc_opf(network_data, Ipopt.Optimizer)

This is Ipopt version 3.14.4, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:       49
Number of nonzeros in inequality constraint Jacobian.:       36
Number of nonzeros in Lagrangian Hessian.............:        3

Total number of variables............................:       21
                     variables with only lower bounds:        0
                variables with lower and upper bounds:       12
                     variables with only upper bounds:        0
Total number of equality constraints.................:       19
Total number of inequality constraints...............:       18
        inequality constraints with only lower bounds:        9
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        9

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  1.2026175e+03 1.25e+00 1.89e+01  -1.0 0.00e+00    -  0.00e+00 0.00e+00  

Dict{String, Any} with 8 entries:
  "solve_time"         => 0.00982594
  "optimizer"          => "Ipopt"
  "termination_status" => LOCALLY_SOLVED
  "dual_status"        => FEASIBLE_POINT
  "primal_status"      => FEASIBLE_POINT
  "objective"          => 5216.03
  "solution"           => Dict{String, Any}("baseMVA"=>100, "branch"=>Dict{Stri…
  "objective_lb"       => -Inf

In [ ]:
# explore some results
dc_results["solution"]["branch"]["1"]
dc_results["solution"]["gen"]["1"]

Dict{String, Any} with 2 entries:
  "qg" => NaN
  "pg" => 0.865645

In [ ]:
# compare total active power production
using Printf
total_gp_ac = sum(res["pg"] for (g,res) in ac_results["solution"]["gen"])
total_gp_dc = sum(res["pg"] for (g,res) in dc_results["solution"]["gen"])
@printf("Total active generation in ACOPF: %.3f\n", total_gp_ac)
@printf("Total active generation in DCOPF: %.3f\n", total_gp_dc)

Total active generation in ACOPF: 3.183
Total active generation in DCOPF: 3.150
